[View in Colaboratory](https://colab.research.google.com/github/nifei/understand-neural-network/blob/master/pytorch.ipynb)

In [1]:
!pip install torch torchvision

 * <b>例子1: </b>首先是对 1x1的tensor x 到 1x1 的 tensor y 的反向求导. loss 取值为 神经元 y的输出的mean, 是一个标量. 

In [21]:
import torch
x = torch.ones(1, 1, requires_grad=True)
print(x)

tensor([[ 1.]])


In [22]:
y = x * x + x
y

tensor([[ 2.]])

In [23]:
out = y.mean()
out

tensor(2.)

In [24]:
out.backward()

In [25]:
x.grad

tensor([[ 3.]])

* <b>例子2:</b> 接着尝试对 多神经元层 到多神经元层 反向求导. x 为 2元tensor, y是二元tensor.  y[i] = f(x[i]), 此处缺图. 

In [40]:
x = torch.FloatTensor([[1, 2]])
x.requires_grad=True
print(x)
y = x * x
print (y)

tensor([[ 1.,  2.]])
tensor([[ 1.,  4.]])


假设目标为"使得y层的神经元都是0", 那么 loss[i]=y[i] - 0, 对 loss 做bp, 运行时错误为:
<code> grad can be implicitly created only for scalar outputs </code>, 因为loss为2x1的tensor. 

In [0]:
loss = y
loss.backward()

但是这里,你照葫芦画瓢, 令loss = y.mean(), 是有问题的. 因为不管优化的目标是x层到y层的weights矩阵, 还是输入层x, 单步优化的幅度对x[0]和x[1] , 或者对不同连接的权重, 肯定是不一样的, x[0]应该减1, x[1]应该减2. y.mean() 显然会丢失这个信息. 

以优化目标为x层到y层的权重为例, 我们期望的, weights 的grad应该是: 

$$[[\frac{dy[0]}{dx[0]}, \frac{dy[0]}{dx[1]}], [\frac{dy[1]}{dx[0]}, \frac{dy[1]}{dx[1]}]]$$

如果优化目标是x层的输入, 我想了一下但是不知道怎么表达, 反正在4个weights固定的情况下, 你是没有办法把输入优化到y层输出完全符合期望的. 

// 举个栗子, y[0] = x[0] + x[1], y[1] = x[0] + x[1] 这样的权重固定的前提下, 你是没有办法, 通过优化 x, 来达到 y = [0, 1] 的目标的. 

所以退而求其次, 只能追求, y和期望的偏差距离最小. 此处用 L1 距离, 也就是y.mean()来作为loss. 


In [41]:
loss = y.mean()
print(loss)
loss.backward()
print(x.grad)

tensor([[ 1.,  2.]])
tensor([[ 1.,  4.]])
tensor(2.5000)
tensor([[ 1.,  2.]])


这个结果还算符合预期, 因为:
$$[[\frac{dy[0]}{dx[0]}, \frac{dy[1]}{dx[1]}]] = 2, 4$$
被平均后各自变成原来的一半也可以理解. 

* <b>例子3:</b>为了验证这个想法, 我们取sum(y)作为loss, 执行下一个单元后看到输出的x.grad是符合预期的 [2, 4]. 

In [42]:
x = torch.FloatTensor([[1, 2]])
x.requires_grad=True
print(x)
y = x * x
print (y)
loss = y.sum()
print(loss)
loss.backward()
print(x.grad)

tensor([[ 1.,  2.]])
tensor([[ 1.,  4.]])
tensor(5.)
tensor([[ 2.,  4.]])


不要高兴的太早, 以上的例子 y[i] = fi(x[i]) 的算子都是一样的, 如果网络不是这样的, 就没那么乐观了. 
* <b>例子4:</b> 此处缺图, x, y 都是2元 tensor, y[0] = x[0] + x[1], y[1] = x[0] - x[1], 所以
$$[[\frac{dy[0]}{dx[0]}, \frac{dy[0]}{dx[1]}], [\frac{dy[1]}{dx[0]}, \frac{dy[1]}{dx[1]}]] = [[1, 1], [1, -1]]$$
显然以下单元的代码执行的结果, x.grad在维度上和结果上都不足够表达梯度求导的信息. 

In [46]:
from torch.autograd import Variable

x = Variable(torch.IntTensor([[1, 2]]), requires_grad=True)
print(x)
y = Variable(torch.zeros(1, 2))
y[0, 0] = x[0, 0] + x[0, 1]
y[0, 1] = x[0, 0] - x[0, 1]
print (y)
loss = y.sum()
print(loss)
loss.backward()
print(x.grad)

tensor([[ 1,  2]], dtype=torch.int32)
tensor([[ 3., -1.]])
tensor(2.)
tensor([[ 2,  0]], dtype=torch.int32)


* <b>例子5: </b> 求单个神经元对输入的梯度. 即以最大/最小化 y[i] 为目标, 优化输入层 x. 
直接考虑是把y[i] 作为loss. 以下代码输出符合预期. 

In [48]:
x = Variable(torch.IntTensor([[1, 2]]), requires_grad=True)
print(x)
y = Variable(torch.zeros(1, 2))
y[0, 0] = x[0, 0] + x[0, 1]
y[0, 1] = x[0, 0] - x[0, 1]
print (y)
loss = y[0][1]
print(loss)
loss.backward()
print(x.grad)


tensor([[ 1,  2]], dtype=torch.int32)
tensor([[ 3., -1.]])
tensor(-1.)
tensor([[ 1, -1]], dtype=torch.int32)


按照pytorch文档的说明, 向量y对向量x的求导, 需要指定向量y各分量的权重(grad_tensors), 然后它做一个加权求和.... 用这个和的标量对向量x求导... ... 
https://pytorch.org/docs/master/autograd.html#torch.autograd.Variable.backward

也就是说, pytorch从根本上, 拒绝向量对向量的求导... 可能是觉得不值得吧. 

* <b>例子6:</b> 还是通过y层的单个神经元的loss来优化x, 只不过通过backward的grad_tensors指定. 这个参数还可以用来对y层全部加权求和, 求平均, 什么的... 以下单元代码输出符合预期, 为y[1]对x[0], x[1]的求导. 

In [56]:
x = Variable(torch.IntTensor([[1, 2]]), requires_grad=True)
print(x)
y = Variable(torch.zeros(1, 2))
y[0, 0] = x[0, 0] + x[0, 1]
y[0, 1] = x[0, 0] - x[0, 1]
print (y)
loss = y
print(loss)
torch.autograd.backward(loss, grad_tensors=torch.FloatTensor([[0, 1]]))
print(x.grad)

tensor([[ 1,  2]], dtype=torch.int32)
tensor([[ 3., -1.]])
tensor([[ 3., -1.]])
tensor([[ 1, -1]], dtype=torch.int32)


无关: 乘法

In [0]:
test0 = torch.zeros(2, 2)
test0[0, 0] = 1
test0[0, 1] = 1
test1 = torch.zeros(2, 2)
test1[0, 0] = 1
test1[1, 0] = 1
print(test0)
print(test1)
print(test0*test1)
print(test0.mm(test1))


In [65]:
def find_non_zero_rect(data, row_count, col_count):
  min_r, min_c, max_r, max_c = None, None, None, None
  for r in range(row_count):
    for c in range(col_count):
      if data[r, c] != 0:
        if min_r is None:
          min_r = r
          max_r = r
        else:
          if min_r > r:
            min_r = r
          if max_r < r:
            max_r = r
        if min_c is None:
          min_c = c
          max_c = c
        else:
          if min_c > c:
            min_c = c
          if max_c < c:
            max_c = c
  if min_r is None or min_c is None or max_r is None or max_c is None:
    return None, None, None, None
  return min_r, min_c, max_r, max_c
    

In [66]:
test0 = torch.zeros(224, 224)
test0[0, 0] = 1
test0[1, 0] = 1
test0[0, 1] = 1
print(test0)
print (find_non_zero_rect(test0, 224, 224))

test0 = torch.zeros(50, 50)
test0[10, 10] = 1
test0[10, 20] = 1
test0[9, 15] = 1
print(test0)
print (find_non_zero_rect(test0, 50, 50))

tensor([[ 1.,  1.,  0.,  ...,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])
(0, 0, 1, 1)
tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])
(9, 10, 10, 20)
